# Prepare raw survey data for annotation

In [1]:
import os

import pandas as pd 
import matplotlib.pyplot as plt
import spacy

In [2]:
data_path = "../data/raw_study_data"
study_timestamp = "2021-03-18_08-40"
study_data = f"data_regard_{study_timestamp}.csv"

In [3]:
data = pd.read_csv(os.path.join(data_path, study_data), index_col=0, encoding='iso-8859-1')
data.head()

,SERIAL,REF,QUESTNNR,MODE,STARTED,SC01,SC08_01,GE02,RN01_CP,RN01,...,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
CASE,,,,,,,,,,,,,,,,,,,,,
198,NaN,NaN,RG,interview,2021-03-03 09:14:28,NaN,NaN,NaN,0.0,female,...,NaN,2021-03-03 09:14:28,0,0,0,0,NaN,NaN,NaN,100
200,NaN,NaN,person_img,interview,2021-03-03 09:14:52,Not answered,NaN,NaN,NaN,NaN,...,NaN,2021-03-03 09:14:56,0,0,2,2,100.0,100.0,3.00,263
202,NaN,NaN,person_img,interview,2021-03-03 09:16:02,Ja,NaN,Ja ich spreche Deutsch auf muttersprachlichem ...,NaN,NaN,...,NaN,2021-03-03 09:17:20,0,0,9,10,75.0,73.0,2.44,315
209,NaN,NaN,person_img,interview,2021-03-03 09:48:26,Not answered,NaN,NaN,NaN,NaN,...,NaN,2021-03-03 09:48:32,0,0,2,2,100.0,100.0,3.00,144
212,NaN,NaN,person_img,interview,2021-03-03 09:50:00,Ja,NaN,Ja ich spreche Deutsch auf muttersprachlichem ...,NaN,NaN,...,NaN,2021-03-03 10:10:20,1,0,26,26,4.0,1.0,0.61,1


In [4]:
def filter_valid(df):
    df = df[df['SC01'] == 'Ja']  # SC01 = Consent
    df = df[df['GE02'].str.startswith('Ja')]  # GE01 = German skills
    df = df.loc[df.loc[:,'PR01_01' : 'NP10_01'].isnull().sum(1) < 20, :]  # at least one sentence
    df = df.drop(['MAILSENT', 'Q_VIEWER', 'MISSING', 'MISSREL', 'TIME_RSI', 'DEG_TIME', 'SERIAL', 'REF'], axis=1)
    df = df.loc[:,~df.columns.str.startswith('TIME')]
    df = df[df['MODE'] != 'admin']
    return df
data = filter_valid(data)

In [5]:
sent_cols = data.loc[:,'PR01_01' : 'NP10_01'].columns
noprompt_cols = data.loc[:, 'NP01_01' : 'NP10_01'].columns
prompt_cols =  data.loc[:, 'PR01_01' : 'PR02_01'].columns


In [6]:
variable_table = f"variables_regard_{study_timestamp}.csv"
variables = pd.read_csv(os.path.join(data_path, variable_table), index_col=0, encoding='iso-8859-1')

In [7]:
def prepare_prompt_dict(variables, cols):
    prompt_list = [p.split(': ')[1].replace('Die Person', 'XYZ') for p in variables.loc[cols, 'LABEL']]
    prompt_dict = dict(zip(cols, prompt_list))
    return prompt_dict

def remove_prompt_duplicate(text, prompt_dict=None):
    # Some participants copied the prompt text into their response
    if prompt_dict is None:
        if 'Die Person' or 'XYZ' in text:
            text = text.replace('Die Person', '')
            text = text.replace('XYZ', '')
    else:
        for val in prompt_dict.values():
            if text.startswith('XYZ') and not val.startswith('XYZ'):
                val = val.replace('XYZ', 'Die Person')
            if val in text:
                text = text.replace(val, '')  
    return text


def get_label_col(df):
    label_dict = {'neutral' : 0, 'positive' : 1, 'negative' : -1}
    return df['FA04'].apply(lambda label : label_dict[label])


def add_BOS_to_reponse(df, cols, prompt_dict=None, case=None):
    texts = pd.DataFrame(columns=cols)
    for col in cols:
        if prompt_dict is None:
            texts[col] = data[col].apply(lambda text :  'XYZ ' + remove_prompt_duplicate(str(text), prompt_dict) if str(text) != 'nan' else None)
        else:
            texts[col] = df[col].apply(lambda text : prompt_dict[col] + ' ' + remove_prompt_duplicate(str(text), prompt_dict) if str(text) != 'nan' else None)
   
    
    texts['label'] = get_label_col(df)
    texts['case'] = df.index
    return texts



In [8]:
def csv_ready(completed_texts):
    # Prepare output file
    texts = []
    labels = []
    case = []
    for _, row in completed_texts.iterrows():
        texts_of_single_case = row[~row.isnull()].tolist()[:-2] # because last two indices of row are label & case
        texts += texts_of_single_case
        labels += [row['label']] * len(texts_of_single_case) 
        case += [row['case']] * len(texts_of_single_case)

    csv_ready = pd.DataFrame(zip(case, labels, texts), columns=['case', 'label', 'text'])
    return csv_ready

prompt_dict = prepare_prompt_dict(variables, prompt_cols)
prompted_texts = add_BOS_to_reponse(data, prompt_cols, prompt_dict)
unprompted_texts = add_BOS_to_reponse(data, noprompt_cols)

csv_ready_prompt = csv_ready(prompted_texts)
csv_ready_noprompt = csv_ready(unprompted_texts)
csv_ready_all = pd.concat([csv_ready_prompt, csv_ready_noprompt]).reset_index()


In [9]:
csv_ready_all['label'].unique()

array([ 0, -1,  1])

In [10]:
csv_ready_all_person = pd.DataFrame(csv_ready_all.apply(lambda text: text['text'].replace('XYZ', 'Die Person'), axis=1),
                        columns=['text'], index=csv_ready_all.index)
csv_ready_all_person

,text
0,Die Person arbeitete als Ballfänger bei einem ...
1,Die Person verdiente Geld indem die anstrengen...
2,Die Person war sehr bekannt dafür häufig herzl...
3,Die Person wurde beschrieben als ruhiger Zeitg...
4,Die Person hatte eine Stelle angefangen als Ba...
...,...
1153,Die Person ist ein Dudelsackspieler.
1154,Die Person arbeitet als Sekretärin.
1155,Die Person ist müde und erschöpft.
1156,Die Person folgt dem Kurs der kommunistischen ...


In [12]:
#dest = os.path.join(data_path, 'for_annotation')
#os.makedirs(dest, exist_ok=True)
#csv_ready_all.to_csv(os.path.join(dest, 'all_with_label.csv'), sep=',', header=True, index=True)
#csv_ready_all_person.to_csv(os.path.join(dest, 'all_without_label.csv'), sep=',', header=True,
# index=True)


